In [ ]:
import pandas as pd
from sqlite3 import connect

: 

In [64]:
results = pd.read_csv("data/results.csv", dtype={'basho': 'string', 'day': 'string', 'rikishi1_id': 'int64', 'rikishi1_rank': 'string', 'rikishi1_shikona': 'string', 'rikishi1_result': 'string', 'rikishi1_win': 'int64', 'kimarite': 'string', 'rikishi2_id': 'int64', 'rikishi2_rank': 'string', 'rikishi2_shikona': 'string', 'rikishi2_result': 'string', 'rikishi2_win': 'string'})
results.head()

,basho,day,rikishi1_id,rikishi1_rank,rikishi1_shikona,rikishi1_result,rikishi1_win,kimarite,rikishi2_id,rikishi2_rank,rikishi2_shikona,rikishi2_result,rikishi2_win
0,1983.01,1,4140,J13w,Chikubayama,0-1 (7-8),0,yorikiri,4306,Ms1e,Ofuji,1-0 (6-1),1
1,1983.01,1,4306,Ms1e,Ofuji,1-0 (6-1),1,yorikiri,4140,J13w,Chikubayama,0-1 (7-8),0
2,1983.01,1,1337,J12w,Tochitsukasa,1-0 (9-6),1,oshidashi,4323,J13e,Shiraiwa,0-1 (3-12),0
3,1983.01,1,4323,J13e,Shiraiwa,0-1 (3-12),0,oshidashi,1337,J12w,Tochitsukasa,1-0 (9-6),1
4,1983.01,1,4097,J12e,Tamakiyama,0-1 (8-7),0,yorikiri,4319,J11w,Harunafuji,1-0 (5-10),1


In [65]:
banzuke = pd.read_csv("data/banzuke.csv", dtype={'basho': 'string', 'id': 'int64', 'rank': 'string', 'rikishi': 'string', 'heya': 'string', 'susshin': 'string', 'birth_date': 'string', 'height': 'float64', 'weight': 'float64', 'birth_date': 'string', 'prev': 'string', 'prev_w': 'float64', 'prev_l': 'float64'})
banzuke.head()

,basho,id,rank,rikishi,heya,shusshin,birth_date,height,weight,prev,prev_w,prev_l
0,1983.01,1354,Y1e,Chiyonofuji,Kokonoe,Hokkaido,1955-06-01,182.0,116.0,Y1e,14.0,1.0
1,1983.01,4080,Y1w,Kitanoumi,Mihogaseki,Hokkaido,1953-05-16,179.0,165.0,Y2eHD,9.0,3.0
2,1983.01,4095,Y2eHD,Wakanohana,Futagoyama,Aomori,1953-04-03,186.0,133.0,Y1w,0.0,0.0
3,1983.01,4104,O1e,Takanosato,Futagoyama,Aomori,1952-09-29,181.0,144.0,O1e,10.0,5.0
4,1983.01,4112,O1w,Kotokaze,Sadogatake,Mie,1957-04-26,183.0,163.0,O1w,10.0,5.0


In [66]:
# Calculate BMI
bmi_list = []

for i in range(len(banzuke)):
    bmi = round(banzuke.loc[i, "weight"] / ((banzuke.loc[i, "height"] / 100) ** 2), 1)
    bmi_list.append(bmi)
bmi_list

banzuke["bmi"] = bmi_list
# Finished calculating BMI

In [67]:
connection = connect(":memory:")
results.to_sql("results", connection)
banzuke.to_sql("banzuke", connection)

def sql(a_string):
    return pd.read_sql(a_string, connection)

In [68]:
combined_sql_query = pd.read_sql_query(
    """SELECT results.basho AS 'basho', results.rikishi1_id AS 'id', results.rikishi1_shikona, banzuke.rikishi as 'rikishi', 
    results.rikishi1_rank AS 'rank', banzuke.height AS 'height', banzuke.weight AS 'weight', banzuke.bmi AS 'bmi', banzuke.heya AS 'heya',
    banzuke.shusshin AS 'shusshin', results.rikishi1_win as 'outcome', results.kimarite AS 'kimarite', results.rikishi2_id AS 'opponent id',
    results.rikishi2_shikona AS 'opponent'
    FROM results 
    JOIN banzuke 
    ON results.rikishi1_id = banzuke.id
    AND results.basho = banzuke.basho""",
    connection,
)

combined_sql_query
combined_df = pd.DataFrame(combined_sql_query)
combined_df = combined_df.drop(["rikishi1_shikona"], axis=1)

combined_df["rank"] = combined_df["rank"].astype(str)

combined_df.to_csv("data/cleaned.csv", index=False)
combined_df.head()

,basho,id,rikishi,rank,height,weight,bmi,heya,shusshin,outcome,kimarite,opponent id,opponent
0,1983.01,4140,Chikubayama,J13w,175.5,115.0,37.3,Miyagino,Fukuoka,0,yorikiri,4306,Ofuji
1,1983.01,4306,Ofuji,Ms1e,193.5,145.0,38.7,Kokonoe,Tokyo,1,yorikiri,4140,Chikubayama
2,1983.01,1337,Tochitsukasa,J12w,178.5,130.0,40.8,Kasugano,Aichi,1,oshidashi,4323,Shiraiwa
3,1983.01,4323,Shiraiwa,J13e,182.4,103.0,31.0,Isegahama,Akita,0,oshidashi,1337,Tochitsukasa
4,1983.01,4097,Tamakiyama,J12e,182.0,134.0,40.5,Kataonami,Fukuoka,0,yorikiri,4319,Harunafuji
